# This should take
- an epitope list (consisting of ID, amino acid sequence) of length _n_
- a _k_ to generate
- an inter-epitope linker (amino acid, hard code for now)
- a scaffold-epitope linker (amino acid; hard code)
- account for specified ORF?

## some math stuff
- _n_ choose _k_ without replacement
# output
- a list of epitope_IDS (i.e. ID-ID-ID), Xmas insert to be cloned into GFP -- the gBlock, basically
- the final GFP-epitope structure for each
- check that final structure does not exceed AAV payload (section between ITRs)
## bonus items
- codon optimization
- a snapgene/DNA scaffold
- restriction cloning mimc

# some biology stuff, i.e. order of events
Workflow is:
1. AA: epitope combos + appropriate linkers
2. codon-optimize (mus musculus, avoid EcoRI + BsrGI sites) to DNA seq
3. Add cloning tails (known; identical) with primer length?
    primers can come from main GFP
4. Output gBlock sequences (how does Genscript want these?)


In [1]:
import itertools
import pandas as pd
from itertools import combinations
import random
from dnachisel import *

In [2]:
def rSubset(arr, r):
    '''return list of all subsets from an array of length r - i.e. from array choose k'''
    # return list of all subsets of length r
    # to deal with duplicate subsets use 
    # set(list(combinations(arr, r)))
    return list(combinations(arr, r))

In [3]:
def triplicator(epitope_sets, repeats=1):
    '''takes a list of single epitopes and generates aa amino acid with # linkrs'''
    triple_set = []
    for ep in epitope_sets:
        new_ep = [(ep) for _ in range(3)]
        triple_set.append(new_ep)
    return triple_set

In [4]:
def makeTag(epitope_sets, linker=None, prepend_linker=None, append_linker=None):
    '''from a list of tuples, combine name sets + epitope sets with specified inter-epitope.
    just some string manip, basically.
    shuffles multi-epitope n choose k setups
    if a prepend is supplied, will add the specified prepend'''
    # collector of new things
    knockin = [] #the tag we're knocking in
    
    # loop the old things
    for epitope_set in epitope_sets:
        # construct name and seq into new lists
        name = []
        seq = []
        
        #shuffler to avoid bias in epitope success
        to_shuffle = list(epitope_set)
        random.shuffle(to_shuffle) 
        epitope_set = tuple(to_shuffle)
        
        # make array
        for i in range(len(epitope_set)):
            name.append(epitope_set[i][0].replace(' ',''))
            seq.append(epitope_set[i][1].strip(' '))

        # construct final name and sequence and append
        name = f'-'.join(name) # name that puppy
        seq = f'{linker}'.join(seq) #make the tag sequence
        
        if prepend_linker:
            seq = prepend_linker + seq
        if append_linker:
            seq = seq + append_linker
        knockin.append((name, seq)) #add to array

    # return new things as a dictionary
    return dict(knockin)

In [5]:
interEp = "GGSGGS" #[GGS]2. for in between epitopes
interScaffEp = "GGSGGS" #linker between eGFP end and beginning of tag

preEpitopee ="ggacgagctgtacaag" #end of eGFP with BsrGI site
endEpitopes = "taagaattcgatatcaag" #stop codon + EcoRI


In [6]:
def read_list(csv, columns = None):
    '''returns records of name + aa seq'''
    epitopes = pd.read_csv(csv)
    records = epitopes[columns].to_records(index=False)
    return records

In [7]:
all_list = read_list('xmas-in-july_tagSet.csv', columns = ['Name', 'AA seq'])
xmas_list = read_list('xmas-in-july_tagSet.csv', columns = ['Name', 'AA seq'])
jun_list = read_list('Human Protein Atlas - under 40 AAs.csv', columns = ['Antibody','Antigen Sequence'])

In [8]:
xmas_sets = rSubset(xmas_list, 3) #combo subset
singlets = rSubset(all_list,1) # just easy
triplicates = triplicator(all_list,3)
jun_triplicates = triplicator(jun_list,3)

tagSets = [xmas_sets, singlets, triplicates, jun_triplicates] # put these together

In [9]:
columns = ['seq_name', 'seq_aa']
df = pd.DataFrame(columns = columns)

for tagSet in tagSets:
    tagDict = makeTag(tagSet,interEp,"GGSGGS") #make amino acid sequences
    new = pd.DataFrame(tagDict.items(), columns = columns)
    df = pd.concat([df, new])#     df = pd.concat(df,pd.DataFrame(tagDict.values()))

,seq_name,seq_aa
0,FLAG-C-Myc-HA,GGSGGSDYKDDDDKGGSGGSEQKLISEEDLGGSGGSYPYDVPDYA
1,FLAG-HA-OLLAS,GGSGGSDYKDDDDKGGSGGSYPYDVPDYAGGSGGSSGFANELGPRLMGK
2,FLAG-StrepII-HA,GGSGGSDYKDDDDKGGSGGSNWSHPQFEKGGSGGSYPYDVPDYA
3,V5-HA-FLAG,GGSGGSGKPIPNPLLGLDSTGGSGGSYPYDVPDYAGGSGGSDYKDDDDK
4,HA-ALFA-FLAG,GGSGGSYPYDVPDYAGGSGGSSRLEEELRRRLTEGGSGGSDYKDDDDK
...,...,...
45,HPA052507-HPA052507-HPA052507,GGSGGSPTPDQKREDDQGAAEIQVPDLEADLQGGSGGSPTPDQKRE...
46,HPA060900-HPA060900-HPA060900,GGSGGSTYRPRPRRSLQPPELIGAMLEPTDEEPKGGSGGSTYRPRP...
47,HPA055941-HPA055941-HPA055941,GGSGGSGRRYRPRRCLRLAQLVGPMLEPSVPEPQGGSGGSGRRYRP...
48,HPA035081-HPA035081-HPA035081,GGSGGSMETVFEEMDEESTGGVSSSKEEIVLGQRLHGGSGGSMETV...


In [ ]:
# # 
### basically it would be nice to call things "GGS-Flagx3" rather than flag flag flag but too bad
# something breaks when _ or - are in the name
# probably needs regex but lil clean up for names

# # initializing string 
# test_str = "HPA052507-HPA052507HPA052507"
  
# # printing original string 
# print("The original string is : " + test_str)
  
# # using list slicing + find()
# # Check if string repeats itself
# res = None
# temp = (test_str + test_str).find(test_str, 1, -1)
# if temp != -1:
#     res = test_str[:temp]

# print(res)
# # print("The root substring of string : " + res)

# codon optimization o'clock
- avoid BsrGI + EcoRI
- optimize for Mus musculus
- append cloning tails

In [ ]:
test = triplet_Tags['FLAG_FLAG_FLAG']

In [ ]:
cloning_RE = [EnzymeSitePattern("BsrGI"), EnzymeSitePattern("EcoRI")]
cloning_RE

In [ ]:
from dnachisel import biotools as bt

In [ ]:
import python_codon_tables as pct
for key in pct.get_all_available_codons_tables().keys():
    print(key)

In [ ]:
seq_test = reverse_translate(test, randomize_codons=True, table = 'Standard')

In [ ]:
def translate(seq):
    start_seq = reverse_translate(test,
                                  randomize_codons=True,
                                  table = 'Standard')

In [ ]:

problem = DnaOptimizationProblem(
    sequence=seq_test,
    constraints=[
        AvoidPattern(cloning_RE[0]), #EcoRI - hacky but oh well, can write iterator later
        AvoidPattern(cloning_RE[1]), #BsrGI
    ],
    objectives=[CodonOptimize(species='m_musculus_domesticus')]
)

# SOLVE THE CONSTRAINTS, OPTIMIZE WITH RESPECT TO THE OBJECTIVE

problem.resolve_constraints()
problem.optimize()

# PRINT SUMMARIES TO CHECK THAT CONSTRAINTS PASS

print(problem.constraints_text_summary())
print(problem.objectives_text_summary())

# GET THE FINAL SEQUENCE (AS STRING OR ANNOTATED BIOPYTHON RECORDS)

final_sequence = problem.sequence  # string
final_record = problem.to_record(with_sequence_edits=True)


In [ ]:
print(final_sequence)